In [76]:
!pip install bs4
!pip3 install ipython 
!pip3 install selenium  
!pip3 install time 
!pip3 install parsel
!pip3 install csv
!pip3 install webdriver_manager

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [77]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep
import csv
from parsel import Selector
from selenium.webdriver.common.keys import Keys
# import web driver
from selenium import webdriver


### Automatic LinkedIn Login

In [78]:
from webdriver_manager.chrome import ChromeDriverManager
#Open a driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com')


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/sfdatabro/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [79]:
#Log in LinkedIn
username = driver.find_element_by_name('session_key')
#Enter your credential username
username.send_keys('email_address')
sleep(0.5)

password = driver.find_element_by_name('session_password')
#Enter your credential password
password.send_keys('password')
sleep(0.5)

# locate submit button by_class_name
sign_in_button = driver.find_element_by_class_name('sign-in-form__submit-btn')
# .click() to mimic button click
sign_in_button.click()
sleep(2)

### Extract desired URLs from Google

**Or enter desired URLs of profiles directly.**

In [80]:
driver.get('https://www.google.com')

In [81]:
#Enter key words
search_query = driver.find_element_by_name('q')
search_query.send_keys('site:linkedin.com/in AND "python developer" AND "London"')
search_query.send_keys(Keys.RETURN)
sleep(0.5)


In [82]:
# Return several URLs as the result of the search on Google
# But need to notice there are advertisements extracted together, so we need to tease them out.
urls = driver.find_elements_by_xpath('//*[@class = "r"]/a[@href]')

# Change it to list type
urls = [url.get_attribute('href') for url in urls]
sleep(0.5)

# urls are now extracted.

### Set a csv file to store the data

In [83]:
writer = csv.writer(open('output.csv', 'w+', encoding='utf-8-sig', newline=''))
writer.writerow(['Name', 'Profile', 'Role', 'Company', 'Location', 'URL'])

40

### For loop to get all the desired data

### Use BeautifulSoup to scrape data

In [84]:
for url in urls:
    driver.get(url)
    sleep(2)
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    name_div = soup.find('div', {'class':'flex-1 mr5'})
    name_loc = name_div.find_all('ul')
    name = name_loc[0].find('li').get_text().strip()
    location = name_loc[1].find('li').get_text().strip()
    profile = name_div.find('h2').get_text().strip()
    exp_ = soup.find('section',{'id':'experience-section'})
    exp = exp_.find('ul').find('div').find('a')
    role = exp.find('h3').get_text().strip()
    company = exp.find_all('p')[1].get_text().strip()
    
    url = driver.current_url
    print(name, profile, role, company, location, url)
    writer.writerow([name, profile, role, company, location, url])

Felix Tillyard Python Developer at Wandera Python Developer Wandera London, United Kingdom https://www.linkedin.com/in/felix-tillyard/?originalSubdomain=uk
Paul Garner Freelance Python Developer at Depop Freelance Python Developer Depop London, Greater London, United Kingdom https://www.linkedin.com/in/pauljgarner/?originalSubdomain=uk
Andrew Namakarskiy Python Developer – IG Python Developer IG London, United Kingdom https://www.linkedin.com/in/namakarskiy/?originalSubdomain=uk
Danny Butler Backend Python Developer at Ostmodern Backend Python Developer Ostmodern London, United Kingdom https://www.linkedin.com/in/danny-butler-62834777/?originalSubdomain=uk
Bartosz Dziendzik Python developer Backend Developer Festicket London, United Kingdom https://www.linkedin.com/in/bartoszdziendzik/?originalSubdomain=uk
Rameshkanth S. Available for new role, Senior Python Developer,  SC Cleared Python Data Engineer Capgemini(Home Office) Tolworth, Greater London, United Kingdom https://www.linkedin.

### Write in csv file

In [ ]:
driver.quit()